# Covid 19 in India

#### Data Cleaning & Transformation

In [1]:
# importing libraries
import numpy as np
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing datasets
df_india = pd.read_csv(r"G:\08 - Github Projects\EDA & Machine Learning\06 - Corona Virus\Raw\covid_19_india.csv")
df_india.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30-01-2020,6:00 PM,Kerala,1,0,0,0,1
1,2,31-01-2020,6:00 PM,Kerala,1,0,0,0,1
2,3,01-02-2020,6:00 PM,Kerala,2,0,0,0,2
3,4,02-02-2020,6:00 PM,Kerala,3,0,0,0,3
4,5,03-02-2020,6:00 PM,Kerala,3,0,0,0,3


In [3]:
# checking info
df_india.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18110 entries, 0 to 18109
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       18110 non-null  int64 
 1   Date                      18110 non-null  object
 2   Time                      18110 non-null  object
 3   State/UnionTerritory      18110 non-null  object
 4   ConfirmedIndianNational   18110 non-null  object
 5   ConfirmedForeignNational  18110 non-null  object
 6   Cured                     18110 non-null  int64 
 7   Deaths                    18110 non-null  int64 
 8   Confirmed                 18110 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.2+ MB


In [4]:
# checking for null values
df_india.isnull().sum()

Sno                         0
Date                        0
Time                        0
State/UnionTerritory        0
ConfirmedIndianNational     0
ConfirmedForeignNational    0
Cured                       0
Deaths                      0
Confirmed                   0
dtype: int64

In [5]:
# checking unique values from State column
df_india['State/UnionTerritory'].unique()

array(['Kerala', 'Telengana', 'Delhi', 'Rajasthan', 'Uttar Pradesh',
       'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra',
       'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand',
       'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh',
       'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh',
       'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands',
       'Goa', 'Unassigned', 'Assam', 'Jharkhand', 'Arunachal Pradesh',
       'Tripura', 'Nagaland', 'Meghalaya',
       'Dadra and Nagar Haveli and Daman and Diu',
       'Cases being reassigned to states', 'Sikkim', 'Daman & Diu',
       'Lakshadweep', 'Telangana', 'Dadra and Nagar Haveli', 'Bihar****',
       'Madhya Pradesh***', 'Himanchal Pradesh', 'Karanataka',
       'Maharashtra***'], dtype=object)

In [6]:
# correcting spellings of few states/ut
state_correction_dict = {
    'Bihar****':'Bihar',
    'Telengana' : 'Telangana',
    'Himanchal Pradesh' : 'Himachal Pradesh',
    'Dadra and Nagar Haveli and Daman and Diu':'Dadra and Nagar Haveli',
    'Daman and Diu' : 'Daman & Diu',
    'Madhya Pradesh***':'Madhya Pradesh',
    'Maharashtra***':'Maharashtra',
    'Karanataka':'Karnataka'
}

def state_correction(state):
    try:
        return state_correction_dict[state]
    except:
        return state
    
# applying function
df_india['State/UnionTerritory'] = df_india['State/UnionTerritory'].apply(state_correction)

# checking again
df_india['State/UnionTerritory'].unique(), df_india['State/UnionTerritory'].nunique()

(array(['Kerala', 'Telangana', 'Delhi', 'Rajasthan', 'Uttar Pradesh',
        'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra',
        'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand',
        'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh',
        'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh',
        'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands',
        'Goa', 'Unassigned', 'Assam', 'Jharkhand', 'Arunachal Pradesh',
        'Tripura', 'Nagaland', 'Meghalaya', 'Dadra and Nagar Haveli',
        'Cases being reassigned to states', 'Sikkim', 'Daman & Diu',
        'Lakshadweep'], dtype=object),
 39)

In [7]:
# converting "Date" to date type
df_india['Date'] = pd.to_datetime(df_india['Date'])

df_india['Date']

0       2020-01-30
1       2020-01-31
2       2020-01-02
3       2020-02-02
4       2020-03-02
           ...    
18105   2021-11-08
18106   2021-11-08
18107   2021-11-08
18108   2021-11-08
18109   2021-11-08
Name: Date, Length: 18110, dtype: datetime64[ns]

In [8]:
# removing unnecessary columns
df_india.drop(columns=["ConfirmedIndianNational", "ConfirmedForeignNational"], inplace=True)

# adding "Active" by subtracting "Cured" & "Deaths" from Confirmed
df_india["Active Cases"] = df_india["Confirmed"] - (df_india["Cured"] + df_india["Deaths"])

##### Calculating Daily values for Confirmed, Deaths, Recovered & Active Cases

In [9]:
# sorting date
df_india.sort_values('Date', inplace=True)

# grouping by
grouped = df_india.groupby("State/UnionTerritory")

# the diff function calculates the difference between the current row and the previous row
daily_cases = grouped[['Confirmed', 'Deaths', 'Cured', 'Active Cases']].diff().fillna(0)

# joining back to the dataframe
df_india = df_india.join(daily_cases, rsuffix='_daily')

In [10]:
df_india

,Sno,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active Cases,Confirmed_daily,Deaths_daily,Cured_daily,Active Cases_daily
2,3,2020-01-02,6:00 PM,Kerala,0,0,2,2,0.0,0.0,0.0,0.0
31,32,2020-01-03,6:00 PM,Kerala,0,0,3,3,1.0,0.0,0.0,1.0
557,558,2020-01-04,7:30 PM,West Bengal,6,3,37,28,0.0,0.0,0.0,0.0
556,557,2020-01-04,7:30 PM,Uttar Pradesh,14,2,103,87,0.0,0.0,0.0,0.0
555,556,2020-01-04,7:30 PM,Uttarakhand,2,0,7,5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17006,17007,2021-12-07,8:00 AM,Himachal Pradesh,198888,3491,203626,1247,5683.0,124.0,10191.0,-4632.0
17007,17008,2021-12-07,8:00 AM,Jammu and Kashmir,311334,4357,318693,3002,12921.0,197.0,27307.0,-14583.0
17008,17009,2021-12-07,8:00 AM,Jharkhand,340798,5119,346328,411,3263.0,37.0,7035.0,-3809.0
17010,17011,2021-12-07,8:00 AM,Kerala,2935423,14586,3065336,115327,362513.0,3782.0,377826.0,-19095.0


In [11]:
# handling negative values
def negtozero(x):
    
    if x < 0:
        return 0
    else:
        return x

In [12]:
# applying
df_india["Confirmed_daily"] = df_india["Confirmed_daily"].apply(negtozero)
df_india["Deaths_daily"] = df_india["Deaths_daily"].apply(negtozero)
df_india["Cured_daily"] = df_india["Cured_daily"].apply(negtozero)
df_india["Active Cases_daily"] = df_india["Active Cases_daily"].apply(negtozero)

In [13]:
df_india

,Sno,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active Cases,Confirmed_daily,Deaths_daily,Cured_daily,Active Cases_daily
2,3,2020-01-02,6:00 PM,Kerala,0,0,2,2,0.0,0.0,0.0,0.0
31,32,2020-01-03,6:00 PM,Kerala,0,0,3,3,1.0,0.0,0.0,1.0
557,558,2020-01-04,7:30 PM,West Bengal,6,3,37,28,0.0,0.0,0.0,0.0
556,557,2020-01-04,7:30 PM,Uttar Pradesh,14,2,103,87,0.0,0.0,0.0,0.0
555,556,2020-01-04,7:30 PM,Uttarakhand,2,0,7,5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17006,17007,2021-12-07,8:00 AM,Himachal Pradesh,198888,3491,203626,1247,5683.0,124.0,10191.0,0.0
17007,17008,2021-12-07,8:00 AM,Jammu and Kashmir,311334,4357,318693,3002,12921.0,197.0,27307.0,0.0
17008,17009,2021-12-07,8:00 AM,Jharkhand,340798,5119,346328,411,3263.0,37.0,7035.0,0.0
17010,17011,2021-12-07,8:00 AM,Kerala,2935423,14586,3065336,115327,362513.0,3782.0,377826.0,0.0


##### Adding latitue and longitude 

In [14]:
df_india.drop(columns="Sno", inplace=True)

# rearranging columns
df_india = df_india[["Date", "State/UnionTerritory",
                     "Confirmed", "Deaths", "Cured", "Active Cases", 
                     "Confirmed_daily", "Deaths_daily", "Cured_daily", "Active Cases_daily"]]

In [15]:
# exporting df_india
df_india.to_csv("Covid 19 - India.csv")